In [1]:
# import builtins

# # Save the original print function
# original_print = print

# # Override the print function
# def custom_print(*args, **kwargs):
#     new_args = []
#     for arg in args:
#         if isinstance(arg, float):
#             # Customize the format here
#             new_args.append(f'{arg:.10f}')
#         else:
#             new_args.append(arg)
#     original_print(*new_args, **kwargs)

# # Override the built-in print with the custom one
# builtins.print = custom_print

In [2]:
import pandas as pd
import os
import gzip
import pickle
import openai
import torch.nn.functional as F
import torch
import re
import copy
from tqdm import tqdm
from openai import OpenAI
client = OpenAI()

In [3]:
import tiktoken
enc = tiktoken.get_encoding("cl100k_base")
# enc.encode("<|endoftext|>",allowed_special={'<|endoftext|>'})
enc.encode(" #")

[674]

In [4]:
def get_openai_response(prompt):
    tokens = 4000
    model="gpt-4-turbo-preview"
    # model="gpt-3.5-turbo-0125"
    response = client.chat.completions.create(
        model=model,
        messages=[
            {
                "role": "system",
                "content": """
The current date is Tuesday, September 10th 2023, 10:30am
You are a helpful assistant.
""".strip()
            },
            {
                "role": "user",
                "content": prompt
            }
            ],
        temperature=0,
        max_tokens=tokens,
        top_p=1,
        # logit_bias={"674": -1.5},
    )
    choice = response.choices[0]

    text = choice.message.content


    return text

In [5]:
hardware_instances = pd.read_csv('hardware_instances.csv') # columns: date,hardware

# add row index
hardware_instances = hardware_instances.reset_index(drop=True)
hardware_instances["row_index"] = hardware_instances.index


In [6]:
unique_values = hardware_instances["hardware"].value_counts().index.tolist()
unique_values.sort()

for a in unique_values:
    print(a)

"+"&"-" screw terminals
"oc dual-x"
#1 computer
#11
#12 2-conductor with ground wire cable
#13
#2
#2 computer
#2 philips bit
#6/#8
#8-32 hex nut
#8-32 x 1-1/4" machine screws
$1.1k ups/battery bank combo
$150 laptop
$16 holmes personal fans
$1600 laptop
$200 item from china
$200 laptops
$25 fans
$400 gpus
$400 rig
$5 evercool single fan
$500 pc
(+5v)
* .batstart batch:cgminer.exe
+ track
-framework opencl
-l/opt/local/lib
-lcurl
-ldl
-lm lib/libgnu.a
-lncurses
-lpthread
-qt client
-v -w256 -f0
.333 ghs sticks
.336 asicminers
.bat file
.bat or .cmd files
.net
.net 3.5 framework
.net 4 framework
.net framework
.net framework v4.0
.net program widget thingie
.vbs script
/dev/ati/card0
/dev/ati/card1
/dev/sda1
/dev/sdx
/dev/sdxx
/dev/ttyacm0
/dev/ttyusb0
/etc/init.d/cgminer
/etc/x11... file
0 asic hashboard
0-0 radeon 6950 900mhz clock average temp 85 degrees 1408 shaders
0-1 radeon 6950 940mhz clock average temp 70 degrees 1408 shaders
0-2 radeon 6570 725mhz clock average temp 70 degrees 

In [7]:
possible_hardware = pd.read_csv('../../hardwarelist/hardware_merged.csv')["hardware_name"].sort_values().tolist()
possible_hardware = [x.lower() for x in possible_hardware]

In [8]:
possible_hardware[:3]

['3410', '4350', '4550']

In [9]:
def have_common_chunk(str1, str2):
    # Iterate through the first string
    for i in range(len(str1) - 1):  # Subtract 1 to avoid index out of range
        # Extract a 2-letter chunk from the first string
        chunk1 = str1[i:i+2]
        
        # Iterate through the second string
        for j in range(len(str2) - 1):  # Subtract 1 to avoid index out of range
            # Extract a 2-letter chunk from the second string
            chunk2 = str2[j:j+2]
            
            # Compare the chunks
            if chunk1 == chunk2:
                return True  # Return True if a common chunk is found

In [ ]:

chunk_length = 50
num_chunks = len(unique_values) // chunk_length + (1 if len(unique_values) % chunk_length != 0 else 0)



# Iterate through the lines chunk_length at a time
# for i in tqdm(range(num_chunks)):
for i in range(num_chunks):
    
    # Calculate start and end indices for slicing
    start_index = i * chunk_length
    end_index = start_index + chunk_length
    chunk = unique_values[start_index:end_index]
    text = "\n".join(chunk).replace("  "," ")

    if not text:  # Make sure the chunk is not empty
        continue
    # check if the file already exists	
    if os.path.exists(f"mappings/hardware_mapping_{i}.txt"):
        continue

    print("processing chunk",i)

    # temp_hardware_lines = "\n".join(possible_hardware)

    # for each hardware name, check if there is at least one string in the chunk with at least 2 letters in common
    # temp_hardware_lines = []
    # for hardware in possible_hardware:
    #     for string in chunk:
    #         if len(set(hardware).intersection(set(string))) >= 2:
    #             temp_hardware_lines.append(hardware)
    #             break

    # for each hardware name, check if there is at least one string in the chunk with at least a 2-gram in common
    temp_hardware_lines = []
    for hardware in possible_hardware:
        for string in chunk:
            if have_common_chunk(hardware, string):
                temp_hardware_lines.append(hardware)
                break

    print(len(temp_hardware_lines))
    temp_hardware_lines = "\n".join(temp_hardware_lines)

    prompt = f"""
User:

Here is a list of hardware that is used for mining bitcoin:

{temp_hardware_lines}



You will be given a collection of strings. Some of them correspond to entries in the list above, while others are unrelated.
Your task is to create a python dictionary that maps the strings from the collection to the names in the list above.
If a string is unclear or not in the table of mining hardware, write "unknown" for the hardware_index value. The names don't need to match exactly.

Example input:
```
sapphire 7950 vapor x
1th/s s2 antminer
1 th mining system
bpmc "blue fury" 2.7 gh/s usb miner v1
18 btc garden v2
antminer s5
block erupter asics
14th s9 of nov.2017 batch
```

Example output:
```python
mapping = {"{"}
    "sapphire 7950 vapor x": "7950",
    "1th/s s2 antminer": "antminer s2",
    "1 th mining system": "unknown",
    "bpmc \\"blue fury\\" 2.7 gh/s usb miner v1": "blue fury",
    "btc garden am-v1", # Assuming a typo or version difference, closest match.
    "antminer s5": "antminer s5",
    "block erupter asics": "unknown",
    "14th s9 of nov.2017 batch": "antminer s9",
{"}"}
```

It is very important that the mapping's inputs correspond exactly to the strings in the collection, including any typo that might be present in the string.
If none of the strings match any hardware names, then write "unknown" for all of them.
In the rare cases where a string can be mapped to a hardware name but the mapping is questionable, write a comment on the same line.

Write the mapping for this collection of strings:
{text}



Assistant:
Sure! Here is the mapping for the collection of strings:
    """.strip()


    print(prompt)
    # print(temp_hardware_lines)
    # print(text)
    print("\n"*10)

    response = get_openai_response(prompt)


    print(response)
    print("\n"*10)

    # save to file
    with open(f"mappings/hardware_mapping_{i}.txt", "w") as file:
        file.write(response)

    # break






In [ ]:
dict_string = ""

for (i,file) in enumerate(os.listdir("mappings")):
    dict_string += "mapping" + str(i) + " = {\n"

    content = open(f"mappings/{file}").read()

    relevant_lines = content.split("```")[1].split("\n")[2:-2]

    comment = content.split("```")[2].strip()

    for line in relevant_lines:
        # # check if there is a hashtag before the colon :
        # if line.split(":")[0].__contains__("#"):
        #     pass
        # else:
        #     # if a line has no comma at the end, add it
        #     if not line.split(":")[1].__contains__(","):
        #         print(line)
        #         if line.split(":")[1].__contains__("#"):
        #             # add a comma before the comment
        #             line = line.split(":")[0] + ": " + line.split(":")[1].split("#")[0].strip() + " # " + line.split("#")[1]
        #         else:
        #             line = line + ","


        dict_string += line + "\n"

    # add the comment
    if comment != "":
        dict_string = dict_string[:-1] + f"# {comment}\n"

    dict_string += "}" + "\n\n"


code = f"""
{dict_string}
def map_hardware_to_table(string):
    res = "not found"
    for i in range(0, {num_chunks}):
        if string in globals()["mapping" + str(i)]:
            res = globals()["mapping" + str(i)][string]
            break
    return res
""".strip()

# save to file
with open("hardware_mapping.py", "w") as file:
    file.write(code)

In [10]:
from hardware_mapping import map_hardware_to_table
hardware_instances = pd.read_csv('hardware_instances.csv') # columns: date,hardware
hardware_instances = hardware_instances.assign(hardware_mapped = hardware_instances["hardware"].apply(map_hardware_to_table))

### Cell below is only for when there's a second mapping

In [11]:
from hardware_mapping2 import map_hardware_to_table2
# apply mapping2 where hardware_mapped is "not found"
for index, row in hardware_instances.iterrows():
    if row["hardware_mapped"] == "not found":
        hardware_instances.at[index, "hardware_mapped"] = map_hardware_to_table2(row["hardware"])

In [12]:
hardware_instances.head(10)

,date,hardware,hardware_mapped
0,2010-11-29 17:21:55,64-bit atom system,unknown
1,2011-01-21 18:36:10,gt330m,unknown
2,2011-01-21 18:36:10,gtx 460,gtx460
3,2011-01-21 18:36:10,8800gtx,unknown
4,2011-01-21 18:36:10,macbook pro,unknown
5,2011-02-19 07:33:30,6 gb of ram,unknown
6,2011-02-19 07:33:30,2x nvidia 1gb gtx 460s,unknown
7,2011-02-19 07:33:30,core i7-950,unknown
8,2011-02-27 21:32:08,6-year old windows desktop,unknown
9,2011-02-27 21:32:08,laptop,unknown


In [13]:
len(hardware_instances)

53399

In [14]:
efficiency = pd.read_csv('../../hardwarelist/hardware_merged.csv') # columns: hardware_name,Mhash/J
efficiency = efficiency.rename(columns={"hardware_name":"hardware_mapped"})

efficiency.head(2)

,hardware_mapped,Mhash/J
0,3410,0.074
1,4350,0.346


In [15]:
joined = hardware_instances.merge(efficiency, on="hardware_mapped", how="left")
joined = joined.dropna(subset=["Mhash/J"])
joined.head(4)

,date,hardware,hardware_mapped,Mhash/J
2,2011-01-21 18:36:10,gtx 460,gtx460,0.4270
11,2011-02-27 21:32:08,5870,5870,1.9060
22,2011-02-28 16:33:47,hd5770,5770,1.9401
23,2011-02-28 16:33:47,hd5570,5570,1.7496


In [16]:
print(len(joined))

14732


In [17]:
table = joined[["date","hardware_mapped","Mhash/J"]]
table = table.rename(columns={"hardware_mapped":"hardware_name"})
table["Mhash/J"] = table["Mhash/J"].astype(float).map(lambda x: x/1000000).map(lambda x: f"{x:.10f}")
table = table.rename(columns={"Mhash/J":"TH/J"})
table = table.sort_values(["date","hardware_name"])
table = table.reset_index(drop=True)
table.head(8)

,date,hardware_name,TH/J
0,2011-01-21 18:36:10,gtx460,0.0000004270
1,2011-02-27 21:32:08,5870,0.0000019060
2,2011-02-28 16:33:47,5570,0.0000017496
3,2011-02-28 16:33:47,5770,0.0000019401
4,2011-03-01 13:33:58,6870,0.0000017264
5,2011-03-01 17:03:07,6870,0.0000017264
6,2011-03-01 17:03:07,9400gt,0.0000000670
7,2011-03-02 03:06:15,4850,0.0000007956


In [18]:
table.to_csv("hardware_instances_with_efficiency.csv", index=False)